# 2.2 Programming an eigenvalue solver

We solve the generalized eigenvalue problem

$$
A u = \lambda M u,
$$

where $A$ comes from $\int \nabla u \nabla v$, and $M$ from $\int u v$, on the space $H_0^1$.

This tutorial shows how to implement linear algebra algorithms.

In [1]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
import math
import scipy.linalg
from scipy import random

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

We setup a stiffness matrix $A$ and a mass matrix $M$, and declare a preconditioner for $A$:

In [2]:
fes = H1(mesh, order=4, dirichlet=".*")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx
pre = Preconditioner(a, "multigrid")

m = BilinearForm(fes)
m += u*v*dx

a.Assemble()
m.Assemble()

u = GridFunction(fes)

The inverse iteration is

$$
u_{n+1} = A^{-1} M u_n,
$$

where the Rayleigh quotient
$$
\rho_n = \frac{\left \langle A u_n, u_n\right \rangle}{\left 
\langle M u_n, u_n\right \rangle}
$$

converges to the smallest eigenvalue $\lambda_1$, with rate of convergence $\lambda_1 / \lambda_2,$ where $\lambda_2$ is the next smallest eigenvalue.

The preconditioned inverse iteration (PINVIT), see [Knyazef+Neymeyr], replaces $A^{-1}$ by an approximate inverse $C^{-1}$:

$$
\rho_n = \frac{\left \langle A u_n, u_n\right \rangle}{\left \langle M u_n, u_n\right \rangle} \\
w_n = C^{-1} (A u_n - \rho_n M u_n) \\
u_{n+1} = u_n + \alpha w_n
$$

The optimal step-size $\alpha$ is found by minimizing the Rayleigh-quotient on a two-dimensional space:

$$
u_{n+1} = \operatorname{arg} \min_{v \in \{ u_n, w_n\}} \frac{\left \langle A v, v\right \rangle}{\left \langle M v, v\right \rangle} 
$$

This minimization problem can be solved by a small eigenvalue problem

$$
a y = \lambda m y
$$
with matrices
$$
a = 
\left( \begin{array}{cc}
\left \langle  A u_n, u_n \right \rangle &
\left \langle  A u_n, w_n \right \rangle \\
\left \langle  A w_n, u_n \right \rangle &
\left \langle  A w_n, w_n \right \rangle 
\end{array} \right), \quad
m = 
\left( \begin{array}{cc}
\left \langle  M u_n, u_n \right \rangle &
\left \langle  M u_n, w_n \right \rangle \\
\left \langle  M w_n, u_n \right \rangle &
\left \langle  M w_n, w_n \right \rangle 
\end{array} \right).
$$

Then, the new iterate is

$$
u_{n+1} = y_1 u_n + y_2 w_n
$$

where $y$ is the eigenvector corresponding to the smaller eigenvalue.

### Implementation in NGSolve

First, we create some help vectors. `CreateVector` creates new vectors of the same format as the existing vector, i.e., same dimension, same real/ complex type, same entry-size, and same MPI-parallel distribution if any.

In [3]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()

Next, we pick a random initial vector, which is  zeroed on the Dirichlet boundary.

Below, the `FV` method (short for `FlatVector`)  lets us access the abstract vector's  linear memory chunk, which in turn provides a "numpy" view of the memory. The projector clears the entries at the Dirichlet boundary:

In [4]:
r.FV().NumPy()[:] = random.rand(fes.ndof)
u.vec.data = Projector(fes.FreeDofs(), True) * r

Finally, we run the PINVIT algorithm. Note that the small matrices $a$ and $m$ defined above are called `asmall` and `msmall` below. They are of type `Matrix`, a class provided by  NGSolve for  dense matrices.

In [5]:
for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotient
    lam = auu/muu
    print (lam / (math.pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    
    eval,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    # print (eval, evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    
Draw (u)

21.81213637425516
2.064573009786206
2.001477129515915
2.0001956761903226
2.000042423766286
2.0000106886506943
2.0000027557306113
2.000000721500192
2.0000001906576643
2.000000050773115
2.0000000136103173
2.00000000368631
2.000000001024078
2.0000000003076437
2.0000000001142872
2.000000000061992
2.0000000000478195
2.0000000000439737
2.000000000042929
2.0000000000426423


## Simultaneous iteration for several eigenvalues

Here are the steps for extending the above to `num` vectors.

Declare a `GridFunction` with multiple components to store several eigenvectors:

In [6]:
num = 5
u = GridFunction(fes, multidim=num)

Create list of help vectors, and a set of random initial vectors in `u`, with zero boundary conditions:

In [7]:
r = u.vec.CreateVector()
Av = u.vec.CreateVector()
Mv = u.vec.CreateVector()

vecs = []
for i in range(2*num):
    vecs.append (u.vec.CreateVector())

for v in u.vecs:
    r.FV().NumPy()[:] = random.rand(fes.ndof)
    v.data = Projector(fes.FreeDofs(), True) * r

Compute `num` residuals, and solve a small eigenvalue problem on a 2 $\times$ `num` dimensional space:

In [8]:
asmall = Matrix(2*num, 2*num)
msmall = Matrix(2*num, 2*num)
lams = num * [1]

for i in range(20):
    
    for j in range(num):
        vecs[j].data = u.vecs[j]
        r.data = a.mat * vecs[j] - lams[j] * m.mat * vecs[j]
        vecs[num+j].data = pre.mat * r

    for j in range(2*num):
        Av.data = a.mat * vecs[j]
        Mv.data = m.mat * vecs[j]
        for k in range(2*num):
            asmall[j,k] = InnerProduct(Av, vecs[k])
            msmall[j,k] = InnerProduct(Mv, vecs[k])

    ev,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    lams[:] = ev[0:num]
    print (i, ":", [lam/math.pi**2 for lam in lams])
    
    for j in range(num):
        u.vecs[j][:] = 0.0
        for k in range(2*num):
            u.vecs[j].data += float(evec[k,j]) * vecs[k]
            
Draw (u)

0 : [10.726438857932433, 62.92913699852923, 86.91000777324052, 105.16826922141368, 117.559352241059]
1 : [2.02436483946409, 8.09353391117313, 9.670043496879813, 16.554924125709093, 22.0739869412405]
2 : [2.001355789776351, 5.1195920754966835, 5.312392369591619, 9.870117591996602, 11.074067398392165]
3 : [2.000191142636381, 5.004577951536769, 5.025783879531015, 8.391475320300788, 10.250808442392456]
4 : [2.0000358312558872, 5.000393109127369, 5.0022184669272045, 8.085675580002757, 10.081305091410346]
5 : [2.0000073283166504, 5.000066175688663, 5.000237688019026, 8.020333656692134, 10.029863042550128]
6 : [2.0000015465455916, 5.000013812106633, 5.0000334378543805, 8.005572893280643, 10.011210401967526]
7 : [2.0000003345738198, 5.000003137721716, 5.000005867892419, 8.001800149101001, 10.004248066600876]
8 : [2.000000073294786, 5.000000743183193, 5.000001186432776, 8.000685694746695, 10.00161311666445]
9 : [2.0000000162424874, 5.000000181234957, 5.0000002618118655, 8.000284765891744, 10.00

The *multidim-component* select in the *Visualization* dialog box allows to switch between the components of the multidim-GridFunction.